In [68]:
import pytreebank
import torch.nn as nn
import torch.nn.functional as F
import re

In [56]:
bank = pytreebank.load_sst("/Users/juanpedrogutierrez/code/sentimentAnalysis/stanfordSentimentTreebank/trees")

## Read vocabulary with regular expresions out of the dictionary.txt file and define the oneHotEncoding function

In [117]:
dictFile = open("../stanfordSentimentTreebank/dictionary.txt")
lines = dictFile.readlines()
exp = r'^(\S+)\|\d+$'
unfilterdWords = map(lambda line: re.findall(exp, line), lines)
words = list(map(lambda x: x[0], filter(lambda x: len(x) > 0, unfilterdWords)))
dictionary = dict((word, number) for number, word in enumerate(words))

In [122]:
def oneHotEncoding(word, dictionary):
    vector = torch.zeros(len(dictionary), 1)
    vector[dictionary[word]] = 1
    return vector

## Define RNTN (Recursive Neural Tensor Network) as a pytorch module

In [61]:
class RNTN(nn.Module):
    
    def __init__(self, vocabularySize, classes = 5, d = 25,):
        self.d = d
        self.L = nn.Linear(vocabularySize, d, bias=False)
        self.W = nn.Linear(d * 2, d)
        self.Ws = nn.Linear(d,  classes)
        self.V = torch.rand(2 * d, 2 * d, d)
    
    def tensorProduct(self, phrase):    
        result = torch.empty(self.d, 1)
        for i in range(self.d):
            result[i] = torch.mm(torch.t(phrase), torch.mm(self.V[:,:,i], phrase))
        return result
    
    def embed(self, inpt):
        return self.L(inpt)
    
    def getSentiment(self, inpt):
        return F.softmax(self.Ws(inpt))
    
    def forward(self, inpt1, inpt2):
        phrase = torch.cat([inpt1, inpt2], dim=0)
        parent = F.tanh(tensorProduc(phrase) + self.W(phrase))
        return parent, self.Ws(parent)